## Read and understand the data

In [1]:
import pandas as pd

data = pd.read_csv('dataset/data.csv')

In [4]:
data

,cap-shape=b,cap-shape=c,cap-shape=f,cap-shape=k,cap-shape=s,cap-shape=x,cap-surface=f,cap-surface=g,cap-surface=s,cap-surface=y,...,population=v,population=y,habitat=d,habitat=g,habitat=l,habitat=m,habitat=p,habitat=u,habitat=w,class
0,F,F,F,F,F,T,F,F,T,F,...,F,F,F,F,F,F,F,T,F,B
1,F,F,F,F,F,T,F,F,T,F,...,F,F,F,T,F,F,F,F,F,A
2,T,F,F,F,F,F,F,F,T,F,...,F,F,F,F,F,T,F,F,F,A
3,F,F,F,F,F,T,F,F,F,T,...,F,F,F,F,F,F,F,T,F,B
4,F,F,F,F,F,T,F,F,T,F,...,F,F,F,T,F,F,F,F,F,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,F,F,F,T,F,F,F,F,T,F,...,F,F,F,F,T,F,F,F,F,A
8120,F,F,F,F,F,T,F,F,T,F,...,T,F,F,F,T,F,F,F,F,A
8121,F,F,T,F,F,F,F,F,T,F,...,F,F,F,F,T,F,F,F,F,A
8122,F,F,F,T,F,F,F,F,F,T,...,T,F,F,F,T,F,F,F,F,B


## Sklearn naive_bayes does not support categorical values in X 

In [5]:
from sklearn.preprocessing import LabelEncoder

X = data.drop(columns=['class'])
y = data['class']
label_encoder = LabelEncoder()
for column in X.columns:
    if X[column].dtype == 'object' and len(X[column].unique()) == 2:
        X[column] = label_encoder.fit_transform(X[column])

## Use Naive Bayes algorithm for binary classification. Use stratified 10-folds cross-validation to measure the performance of the algorithm.
## 10-folds means in each iteration of cross-validation, the model is trained on 9*(dataset_size/10) samples and rest will be using on test

## To measure performance metrics I will store TP, TN, FP, FN in each stratified k-fold iteration

In [20]:
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB


nb_classifier = GaussianNB()


# use n_splits = 10 for 10-folds
# Initialize stratified 10-fold cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Parameters for calculating macro-average metrics
tn_A, fp_A, fn_A, tp_A = 0,0,0,0
tn_B, fp_B, fn_B, tp_B = 0,0,0,0


# Initialize variables for accumulating macro-average metrics for class A
total_fold_accuracy_macro_A = 0
total_fold_precision_macro_A = 0
total_fold_recall_macro_A = 0
total_fold_tn_rate_macro_A = 0
total_fold_f1_macro_A = 0

# Initialize variables for accumulating macro-average metrics for class B
total_fold_accuracy_macro_B = 0
total_fold_precision_macro_B = 0
total_fold_recall_macro_B = 0
total_fold_tn_rate_macro_B = 0
total_fold_f1_macro_B = 0

# Parameters for calculating micro-average metrics
total_acc ,total_prec ,total_rec ,total_tnrate ,total_f1 = 0,0,0,0,0
total_tp, total_tn, total_fp, total_fn = 0,0,0,0

total_acc_A ,total_prec_A ,total_rec_A ,total_tnrate_A ,total_f1_A = 0,0,0,0,0
total_acc_B ,total_prec_B ,total_rec_B ,total_tnrate_B ,total_f1_B = 0,0,0,0,0

# Iterate over each fold
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Train Naive Bayes classifier
    nb_classifier.fit(X_train, y_train)
    
    # Predict on test data
    y_pred = nb_classifier.predict(X_test)
    
    
    # tn means the model predicted the negative and the actual label was also negative
    # fp means the model predicted the positive and the actual label was negative
    # fn means the model predicted the negative and the actual label was positive
    # tn means the model predicted the negative and the actual label was also negative

    # our negative class is B
    # our positive class is A
    tn_A = sum((y_pred == 'B')&(y_test == 'B'))
    fp_A = sum((y_pred == 'A')&(y_test == 'B'))
    fn_A = sum((y_pred == 'B')&(y_test == 'A'))
    tp_A = sum((y_pred == 'A')&(y_test == 'A'))
    
    # our negative class is A
    # our positive class is B
    tn_B = sum((y_pred == 'A')&(y_test == 'A'))
    fp_B = sum((y_pred == 'B')&(y_test == 'A'))
    fn_B = sum((y_pred == 'A')&(y_test == 'B'))
    tp_B = sum((y_pred == 'B')&(y_test == 'B'))

    # Calculate each iteration's macro average metrics
    # First calculate for class A
    accuracy_A = (tp_A + tn_A) / (tp_A + tn_A + fp_A + fn_A)
    precision_A = tp_A / (tp_A + fp_A)
    recall_A = tp_A / (tp_A + fn_A)
    tn_rate_A = tn_A / (tn_A + fp_A)
    f1_A = 2 * (precision_A * recall_A) / (precision_A + recall_A)

    # Calculate for class B
    accuracy_B = (tp_B + tn_B) / (tp_B + tn_B + fp_B + fn_B)
    precision_B = tp_B / (tp_B + fp_B)
    recall_B = tp_B / (tp_B + fn_B) 
    tn_rate_B = tn_B / (tn_B + fp_B)
    f1_B = 2 * (precision_B * recall_B) / (precision_B + recall_B)

    # Accumulate macro-average metrics for each class
    total_fold_accuracy_macro_A += accuracy_A
    total_fold_precision_macro_A += precision_A
    total_fold_recall_macro_A += recall_A
    total_fold_tn_rate_macro_A += tn_rate_A
    total_fold_f1_macro_A += f1_A
    
    total_fold_accuracy_macro_B += accuracy_B
    total_fold_precision_macro_B += precision_B
    total_fold_recall_macro_B += recall_B
    total_fold_tn_rate_macro_B += tn_rate_B
    total_fold_f1_macro_B += f1_B

# Calculate macro average metrics
# Calculate macro-average metrics by averaging across all folds
accuracy_macro_A = total_fold_accuracy_macro_A / 10
precision_macro_A = total_fold_precision_macro_A / 10
recall_macro_A = total_fold_recall_macro_A / 10
tn_rate_macro_A = total_fold_tn_rate_macro_A / 10
f1_macro_A = total_fold_f1_macro_A / 10

accuracy_macro_B = total_fold_accuracy_macro_B / 10
precision_macro_B = total_fold_precision_macro_B / 10
recall_macro_B = total_fold_recall_macro_B / 10
tn_rate_macro_B = total_fold_tn_rate_macro_B / 10
f1_macro_B = total_fold_f1_macro_B / 10

# Combine the macro-average metrics for class A and class B
accuracy_macro = (accuracy_macro_A + accuracy_macro_B) / 2
precision_macro = (precision_macro_A + precision_macro_B) / 2
recall_macro = (recall_macro_A + recall_macro_B) / 2
tn_rate_macro = (tn_rate_macro_A + tn_rate_macro_B) / 2
f1_macro = (f1_macro_A + f1_macro_B) / 2

print("Macro Averages:")
print("Accuracy: ",accuracy_macro)
print("TPrate (Recall): " ,recall_macro)
print("TNrate: ", tn_rate_macro)
print("Precision: ",precision_macro)
print("F-Score: " ,f1_macro)


Macro Averages:
Accuracy:  0.9438706911699659
TPrate (Recall):  0.9457895952969642
TNrate:  0.9457895952969642
Precision:  0.947778909104917
F-Score:  0.9438461751393117


# Calculate evaluation metrics
    
    # accuracy = (tp + tn) / (tp + tn + fp + fn)
    # precision = tp / (tp + fp)
    # recall = tp / (tp + fn) 
    # tn_rate = tn / (tn + fp)
    # f1 = 2 * (precision * recall) / (precision + recall)